In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
import numpy as np
import datetime
import scipy.misc



# 1.Generative Adversarial Network(GAN)

* Architecture of GAN
<div align="center">
    <img src="https://cdn-images-1.medium.com/v2/resize:fit:2000/1*39Nnni_nhPDaLu9AnTLoWw.png",alt="GAN">
</div>

* Generator
<div align="center">
    <img src="https://cdn-images-1.medium.com/v2/resize:fit:1000/1*7i9iCdLZraZkrMy1-KADrA.png",alt="Generator">
</div>

* Discriminator
<div align="center">
    <img src="https://www.researchgate.net/profile/Sinan-Kaplan/publication/319093376/figure/fig20/AS:526859935731712@1502624605127/Architecture-of-proposed-discriminator-network-which-is-part-of-GAN-based-on-CNN-units.png",alt="Discriminator">
</div>

In [2]:
Model_Name='ConditionGAN'
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
def to_cuda(x):
    return x.to(device)

In [4]:
def to_onehot(x,num_classes=10):
    assert isinstance(x,int) or isinstance(x,(torch.LongTensor,torch.cuda.LongTensor))
    if isinstance(x,int):
        c=torch.zeros(1,num_classes).long()
        c[0][x]=1
    else:
        x=x.cpu()
        c=torch.LongTensor(x.size(0),num_classes)
        c.zero_()
        c.scatter_(1,x,1)
    return c

In [5]:
def get_simple_image(G,n_noise=100):
    for num in range(10):
        c=to_cuda(to_onehot(num))
        for i in range(10):
            z=to_cuda(torch.randn(1,n_noise))
            y_hat=G(z,c)
            line_img=torch.cat((line_img,y_hat.view(28,28)),dim=1) if i>0 else y_hat.view(28,28)
        all_img=torch.cat((all_img,line_img),dim=0) if num>0 else line_img
    img=all_img.cpu().data.numpy()
    return img

    

In [9]:
class Discriminator(nn.Module):
    
    def __init__(self,input_size=784,label_size=10,num_classes=1):
        super(Discriminator,self).__init__()
        self.layer1=nn.Sequential(
            nn.Linear(input_size+label_size,200),
            nn.ReLU(),
            nn.Dropout()
        )
        self.layer2=nn.Sequential(
            nn.Linear(200,200),
            nn.ReLU(),
            nn.Dropout()
        )
        self.layer3=nn.Sequential(
            nn.Linear(200,num_classes),
            nn.Sigmoid()
        )
    
    def forward(self,x,y):
        x,y=x.view(x.size(0),-1),y.view(y.size(0),-1).float()
        v=torch.cat((x,y),1)
        y_=self.layer1(v)
        y_=self.layer2(y_)
        y_=self.layer3(y_)
        return y_

In [10]:
class Generator(nn.Module):

    def __init__(self,input_size=100,label_size=10,num_classes=784):
        super(Generator,self).__init__()
        self.layer=nn.Sequential(
            nn.Linear(input_size+label_size,200),
            nn.LeakyReLU(0.2),
            nn.Linear(200,200),
            nn.LeakyReLU(0.2),
            nn.Linear(200,num_classes),
            nn.Tanh()
        )
    
    def forward(self,x,y):
        x,y=x.view(x.size(0),-1),y.view(y.size(0),-1).float()
        v=torch.cat((x,y),1)
        y_=self.layer(v)
        y_=y_.view(x.size(0),1,28,28)
        return y_

In [11]:
D=to_cuda(Discriminator())
G=to_cuda(Generator())

In [12]:
transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.5,0.5,0.5),
     std=(0.5,0.5,0.5))]
)

In [14]:
mnist=datasets.MNIST(root='./data/',train=True,transform=transform,download=True)

In [15]:
batch_size=64
condition_size=10

In [18]:
data_loader=DataLoader(dataset=mnist,batch_size=batch_size,shuffle=True,drop_last=True)

In [19]:
criterion=nn.BCELoss()
D_opt=torch.optim.Adam(D.parameters())
G_opt=torch.optim.Adam(G.parameters())

In [20]:
max_epoch=100
step=0
n_critic=5
n_noise=100


In [21]:
D_labels=to_cuda(torch.ones(batch_size))
D_fakes=to_cuda(torch.zeros(batch_size))

In [ ]:
for epoch in range(max_epoch):
    for idx,(image,labels) in enumerate(data_loader):
        step+=1

        x=to_cuda(image)
        y=labels.view(batch_size,1)
        y=to_cuda(to_onehot(y))
        x_outputs=D(x,y)
        D_x_loss=criterion(x_outputs,D_labels)

        z=to_cuda(torch.randn(batch_size,n_noise))
        z_output=D(G(z,y),y)
        